In [1]:
#import ds tools
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.model_selection import StratifiedKFold

from IPython.display import display
import re




# Sentiment Analysis Pipeline
This code implements a sentiment analysis pipeline using Natural Language Toolkit (NLTK) and scikit-learn libraries in Python. The pipeline performs the following steps:

Data pre-processing: The raw text data is pre-processed using various techniques, such as removing special characters and extra spaces, converting to lowercase, and lemmatizing the words.
Feature extraction: The pre-processed text data is converted to a numerical feature matrix using the Term Frequency-Inverse Document Frequency (TF-IDF) algorithm.
Model training and evaluation: Several machine learning models are trained and evaluated using the pre-processed and feature-extracted data. The models include Logistic Regression, Linear Support Vector Classification, Naive Bayes, Random Forest, Decision Tree, K-Nearest Neighbors, Support Vector Machines, Gradient Boosting, AdaBoost, Gaussian Process, and Gaussian Naive Bayes. The performance of each model is evaluated using k-fold cross-validation and various evaluation metrics, such as accuracy, precision, recall, and F1-score.
The code is organized as follows:

## Importing libraries
The code begins by importing the necessary libraries, including NLTK, scikit-learn, pandas, and re. The NLTK library is used for lemmatization, and the scikit-learn library is used for feature extraction, model training, and evaluation. Pandas is used for data processing, and re is used for regular expression operations.

## Data pre-processing
The pre-processing of the text data is performed using the clean_text() function, which removes special characters and extra spaces, converts the text to lowercase, and lemmatizes the words using the NLTK library. The pre-processing function is then applied to the training and testing data using the preprocess_text() wrapper function.

## Model training and evaluation
The models are defined as a dictionary of scikit-learn classifiers, including Logistic Regression, Linear Support Vector Classification, Naive Bayes, Random Forest, Decision Tree, K-Nearest Neighbors, Support Vector Machines, Gradient Boosting, AdaBoost, Gaussian Process, and Gaussian Naive Bayes.

The classifiers are trained and evaluated using k-fold cross-validation with 3 splits. The performance of each model is evaluated using various evaluation metrics, including accuracy, precision, recall, and F1-score. The results of the evaluation are stored in a Pandas DataFrame and displayed.

The best-performing classifier is selected based on the F1-score, and its hyperparameters are tuned using Bayesian optimization. The best hyperparameters are then used to create a final model, which is trained on the entire training set and evaluated on the testing set.

The performance of the final model is evaluated using various evaluation metrics, including accuracy, precision, recall, and F1-score, and the results are displayed.

In [2]:
#load data
x_train = pd.read_csv("./Dataset/x_train.csv",header=None,names=['website','text'])
y_train = pd.read_csv("./Dataset/y_train.csv",header=None,names=['positive'])
x_test = pd.read_csv("./Dataset/x_test.csv",header=None,names=['website',"text"])
y_test = pd.read_csv("./Dataset/y_test.csv",header=None,names=['positive'])

print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)
display(x_train.head())
display(y_train.head())

(2400, 2) (2400, 1) (600, 2) (600, 1)


,website,text
0,amazon,Oh and I forgot to also mention the weird colo...
1,amazon,THAT one didn't work either.
2,amazon,Waste of 13 bucks.
3,amazon,"Product is useless, since it does not have eno..."
4,amazon,None of the three sizes they sent with the hea...


,positive
0,0
1,0
2,0
3,0
4,0


# Task 2 : Text representation

after we load the data, we need to clean it. we need to remove the stop words and the punctuation. we also need to stem the words. we will use the nltk library to do that.

re is used to remove the punctuation and special character.
WordnetLemmatizer is used to stem the words.

then we will create pipeline to vectorize the data and do the classification.

we will use the following classifiers:
```python
#import all of the models
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, NuSVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
classifiers = {
    "LogisiticRegression": LogisticRegression(),
    "LinearSVC": LinearSVC(),
    "MultinomialNB": MultinomialNB(),
    "BernoulliNB": BernoulliNB(),
    "SGDClassifier": SGDClassifier(),
    "RandomForestClassifier": RandomForestClassifier(),
    "DecisionTreeClassifier": DecisionTreeClassifier(),
    "KNeighborsClassifier": KNeighborsClassifier(),
    "SVC": SVC(),
    "NuSVC": NuSVC(),
    "GaussianProcessClassifier": GaussianProcessClassifier(),
    "GradientBoostingClassifier": GradientBoostingClassifier(),
    "AdaBoostClassifier": AdaBoostClassifier(),
    "GaussianNB": GaussianNB(),
}

This code is a Python script for performing text classification using various machine learning classifiers. The script uses the Natural Language Toolkit (NLTK) library for lemmatization and the scikit-learn library for the machine learning classifiers and feature extraction. The dataset used for classification is assumed to be stored in two pandas dataframes, x_train and y_train for training data and x_test and y_test for testing data.

The script performs the following steps:

1. Imports necessary libraries: nltk, sklearn, pandas, re
2. Downloads necessary packages from nltk.
3. Defines a function lemmatize() that lemmatizes text using the WordNetLemmatizer from nltk.
4. Defines a function clean_text() that cleans raw text by removing special characters, extra spaces, and lowercasing the text, and then calls the lemmatize() function to lemmatize the text.
5. Defines a function preprocess_text() that applies the clean_text() function to a pandas Series of text data.
6. Preprocesses the text data in the x_train and x_test dataframes using the preprocess_text() function.
7. Defines a dictionary of classifiers to use for training and testing.
8. Defines evaluation metrics, including accuracy, precision, recall, and f1-score.
9. Loops over each classifier in the dictionary and performs a stratified k-fold cross-validation for training and testing. The pipeline includes a TfidfVectorizer for feature extraction, a FunctionTransformer to convert the sparse matrix output to a dense array, and the classifier being tested.
10. Evaluates each classifier and stores the results in a dictionary that includes the classifier name and the evaluation metrics.
11. Converts the results to a pandas dataframe, groups the results by classifier, calculates the mean evaluation metrics for each classifier, sorts the results by the f1-score, and displays the classifier with the highest f1-score.

In [3]:
import nltk
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.svm import SVC

#download nltk packages
nltk.download('wordnet') 
nltk.download('averaged_perceptron_tagger')

lemmatizer = WordNetLemmatizer()
tfidf = TfidfVectorizer()

def lemmatize(text):
    #lemmatize text
    text = ' '.join([lemmatizer.lemmatize(w) for w in text.split()])
    return text
def clean_text(text):
    #clean raw text
    text_regex = [
        (r'[^a-zA-Z0-9\s]', ' '), #remove special characters
        (r'\s+', ' '), #remove extra spaces
    ]
    for regex, replace in text_regex:
        text = re.sub(regex, replace, text)
    #lowercase
    text = text.lower()
    #lemmatize
    text = lemmatize(text)
    return text


#preprocess text wrappwer for dataframe
def preprocess_text(col: pd.Series) -> pd.Series:
    return col.apply(clean_text)


#preprocess text
x_train['text_pre'] = preprocess_text(x_train['text'])
x_test['text_pre'] = preprocess_text(x_test['text'])

# defining classifiers
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, NuSVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
classifiers = {
    "LogisiticRegression": LogisticRegression(),
    "LinearSVC": LinearSVC(),
    "MultinomialNB": MultinomialNB(),
    "BernoulliNB": BernoulliNB(),
    "SGDClassifier": SGDClassifier(),
    "RandomForestClassifier": RandomForestClassifier(),
    "DecisionTreeClassifier": DecisionTreeClassifier(),
    "KNeighborsClassifier": KNeighborsClassifier(),
    "SVC": SVC(),
    "NuSVC": NuSVC(),
    "GaussianProcessClassifier": GaussianProcessClassifier(),
    "GradientBoostingClassifier": GradientBoostingClassifier(),
    "AdaBoostClassifier": AdaBoostClassifier(),
    "GaussianNB": GaussianNB(),
}

# selecting baseline classifier to use for parameter tuning
#defining evaluation metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import FunctionTransformer



results = []

for key, classifier in classifiers.items():

    # stratify kfold
    skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
    for train_index, test_index in skf.split(x_train['text_pre'], y_train['positive']):
        x_train_fold, x_test_fold = x_train['text_pre'][train_index], x_train['text_pre'][test_index]
        y_train_fold, y_test_fold = y_train['positive'][train_index], y_train['positive'][test_index]
        #create pipeline
        pipeline = Pipeline([
            ('tfidf', TfidfVectorizer()),
            ("todense", FunctionTransformer(lambda x: x.toarray(), accept_sparse=True)),
            ('clf', classifier),
        ])
        #fit pipeline
        pipeline.fit(x_train_fold, y_train_fold)
        #predict
        y_pred = pipeline.predict(x_test_fold)
        #evaluate
        results.append({
            "classifier": key,
            "accuracy": accuracy_score(y_test_fold, y_pred),
            "precision": precision_score(y_test_fold, y_pred),
            "recall": recall_score(y_test_fold, y_pred),
            "f1": f1_score(y_test_fold, y_pred),

        })
   
#convert results to dataframe
results = pd.DataFrame(results)
# results
res = results.groupby("classifier").mean().sort_values("f1", ascending=False)
display(res)
display(res.idxmax())


[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/suchattangjarukij/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/suchattangjarukij/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


# parameter tuning
This next block of code performs hyperparameter tuning for a classifier using the Optuna library. The objective function is defined as objective(trial) and takes a trial object as its input. The function first defines a set of hyperparameters to optimize, which includes the degree of the polynomial kernel, the decision function shape, the kernel type, and the random state. The StratifiedKFold method is used to split the data into three folds for cross-validation, and the Pipeline method is used to create a pipeline consisting of a TfidfVectorizer, a NuSVC classifier, and some hyperparameters to optimize.

The pipeline.set_params(**parameters) method is called to set the values of the hyperparameters defined in the objective function. The pipeline.fit(x_train_fold, y_train_fold) method is then used to fit the pipeline to the training data, and the pipeline.predict(x_test_fold) method is used to predict the class labels of the test data. The f1_score(y_test_fold, y_pred) method is used to compute the F1 score of the predictions, and the mean F1 score of the three folds is returned as the objective value.

The optuna.create_study(direction="maximize") method is used to create a study object, and the study.optimize(objective, n_trials=50,timeout=300,n_jobs=-1) method is used to run 50 trials with a maximum time of 300 seconds per trial in parallel using all available cores. The objective of the hyperparameter tuning is to find the combination of hyperparameters that will maximize the F1 score of the classifier.

In [ ]:
# tune parameters for best classifier using optuna
import optuna

def objective(trial):
    # define parameters to tune
    parameters = {
        # "tfidf__ngram_range": trial.suggest_categorical("tfidf__ngram_range", [(1,1), (1,2), (1,3)]),
        # "tfidf__max_df": trial.suggest_float("tfidf__max_df", 0.5, 1.0),
        # "tfidf__min_df": trial.suggest_float("tfidf__min_df", 0.0, 0.5),
        "clf__degree": trial.suggest_int("clf__degree", 1, 3),
        "clf__decision_function_shape": trial.suggest_categorical("clf__decision_function_shape", ["ovo", "ovr"]),
        "clf__kernel": trial.suggest_categorical("clf__kernel", ["linear", "poly", "rbf", "sigmoid"]),
        "clf__random_state": 42,
    }
    # create pipeline for each stratified kfold
    skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
    f1 = []
    for train_index, test_index in skf.split(x_train['text_pre'], y_train['positive']):
        x_train_fold, x_test_fold = x_train['text_pre'][train_index], x_train['text_pre'][test_index]
        y_train_fold, y_test_fold = y_train['positive'][train_index], y_train['positive'][test_index]
        # create pipeline

        pipeline = Pipeline([
            ('tfidf', TfidfVectorizer()),
            ("todense", FunctionTransformer(lambda x: x.toarray(), accept_sparse=True)),
            ('clf', NuSVC()),
        ])
        # fit pipeline
        pipeline.set_params(**parameters)
        pipeline.fit(x_train_fold, y_train_fold)
        # predict
        y_pred = pipeline.predict(x_test_fold)
        # evaluate
        f1.append(f1_score(y_test_fold, y_pred))

    return np.mean(f1)

# create study
study = optuna.create_study(direction="maximize")
# optimize
study.optimize(objective, n_trials=50,timeout=300,n_jobs=-1)

[I 2023-03-24 14:12:09,296] A new study created in memory with name: no-name-88094506-bcee-4acd-ae4e-ee1bdf2bef44
[I 2023-03-24 14:13:03,428] Trial 5 finished with value: 0.8052212179274479 and parameters: {'clf__degree': 2, 'clf__decision_function_shape': 'ovr', 'clf__kernel': 'linear'}. Best is trial 5 with value: 0.8052212179274479.
[I 2023-03-24 14:13:03,485] Trial 7 finished with value: 0.8052212179274479 and parameters: {'clf__degree': 3, 'clf__decision_function_shape': 'ovo', 'clf__kernel': 'linear'}. Best is trial 5 with value: 0.8052212179274479.
[I 2023-03-24 14:13:03,546] Trial 6 finished with value: 0.8052212179274479 and parameters: {'clf__degree': 1, 'clf__decision_function_shape': 'ovo', 'clf__kernel': 'linear'}. Best is trial 5 with value: 0.8052212179274479.
[I 2023-03-24 14:13:03,649] Trial 1 finished with value: 0.8052212179274479 and parameters: {'clf__degree': 3, 'clf__decision_function_shape': 'ovo', 'clf__kernel': 'linear'}. Best is trial 5 with value: 0.80522121

In [ ]:
study.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_clf__decision_function_shape,params_clf__degree,params_clf__kernel,state
0,0,0.805221,2023-03-24 14:12:09.297906,2023-03-24 14:13:03.695262,0 days 00:00:54.397356,ovr,1,linear,COMPLETE
1,1,0.805221,2023-03-24 14:12:09.300124,2023-03-24 14:13:03.648673,0 days 00:00:54.348549,ovo,3,linear,COMPLETE
2,2,0.805221,2023-03-24 14:12:09.302599,2023-03-24 14:13:03.715107,0 days 00:00:54.412508,ovo,1,linear,COMPLETE
3,3,0.814808,2023-03-24 14:12:09.303578,2023-03-24 14:13:15.819977,0 days 00:01:06.516399,ovr,2,poly,COMPLETE
4,4,0.815984,2023-03-24 14:12:09.304461,2023-03-24 14:13:28.482208,0 days 00:01:19.177747,ovo,2,rbf,COMPLETE
5,5,0.805221,2023-03-24 14:12:09.305519,2023-03-24 14:13:03.427819,0 days 00:00:54.122300,ovr,2,linear,COMPLETE
6,6,0.805221,2023-03-24 14:12:09.306347,2023-03-24 14:13:03.545645,0 days 00:00:54.239298,ovo,1,linear,COMPLETE
7,7,0.805221,2023-03-24 14:12:09.307620,2023-03-24 14:13:03.484996,0 days 00:00:54.177376,ovo,3,linear,COMPLETE
8,8,0.805221,2023-03-24 14:13:03.441394,2023-03-24 14:14:02.071313,0 days 00:00:58.629919,ovo,2,linear,COMPLETE
9,9,0.800377,2023-03-24 14:13:03.494640,2023-03-24 14:14:00.159965,0 days 00:00:56.665325,ovo,1,sigmoid,COMPLETE


## Best Model with Hyperparameter from optuna

In [ ]:
best_params = study.best_params

# create pipeline with best parameters
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ("todense", FunctionTransformer(lambda x: x.toarray(), accept_sparse=True)),
    ('clf', NuSVC()),
])
pipeline.set_params(**best_params,clf__probability=True)
pipeline.fit(x_train['text_pre'], y_train['positive'])
y_pred = pipeline.predict(x_test['text_pre'])



              precision    recall  f1-score   support

           0       0.80      0.89      0.84       300
           1       0.87      0.77      0.82       300

    accuracy                           0.83       600
   macro avg       0.83      0.83      0.83       600
weighted avg       0.83      0.83      0.83       600

accuracy:  0.83
precision:  0.8721804511278195
recall:  0.7733333333333333
f1:  0.8197879858657243


In [ ]:
print(classification_report(y_test['positive'], y_pred))


print("accuracy: ", accuracy_score(y_test['positive'], y_pred))
print("precision: ", precision_score(y_test['positive'], y_pred))
print("recall: ", recall_score(y_test['positive'], y_pred))
print("f1: ", f1_score(y_test['positive'], y_pred))
print("confusion matrix: ", confusion_matrix(y_test['positive'], y_pred))

              precision    recall  f1-score   support

           0       0.80      0.89      0.84       300
           1       0.87      0.77      0.82       300

    accuracy                           0.83       600
   macro avg       0.83      0.83      0.83       600
weighted avg       0.83      0.83      0.83       600

accuracy:  0.83
precision:  0.8721804511278195
recall:  0.7733333333333333
f1:  0.8197879858657243
confusion matrix:  [[266  34]
 [ 68 232]]


## TF-IDF representation explained

In [ ]:
# explaining tfidf vectorizer from pipeline
# get feature names
feature_names = pipeline.named_steps['tfidf'].get_feature_names()
# get coefficients
coef = pipeline.named_steps['clf'].coef_
# get top 10 features
top10 = np.argsort(coef[0])[-10:]
# get bottom 10 features
bottom10 = np.argsort(coef[0])[:10]
# print top 10 features
print("top 10 features")
for i in top10:
    print(feature_names[i])
# print bottom 10 features
print("bottom 10 features")
for i in bottom10:
    print(feature_names[i])



In [ ]:
study.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_clf__decision_function_shape,params_clf__degree,params_clf__kernel,state
0,0,0.805221,2023-03-24 14:12:09.297906,2023-03-24 14:13:03.695262,0 days 00:00:54.397356,ovr,1,linear,COMPLETE
1,1,0.805221,2023-03-24 14:12:09.300124,2023-03-24 14:13:03.648673,0 days 00:00:54.348549,ovo,3,linear,COMPLETE
2,2,0.805221,2023-03-24 14:12:09.302599,2023-03-24 14:13:03.715107,0 days 00:00:54.412508,ovo,1,linear,COMPLETE
3,3,0.814808,2023-03-24 14:12:09.303578,2023-03-24 14:13:15.819977,0 days 00:01:06.516399,ovr,2,poly,COMPLETE
4,4,0.815984,2023-03-24 14:12:09.304461,2023-03-24 14:13:28.482208,0 days 00:01:19.177747,ovo,2,rbf,COMPLETE
5,5,0.805221,2023-03-24 14:12:09.305519,2023-03-24 14:13:03.427819,0 days 00:00:54.122300,ovr,2,linear,COMPLETE
6,6,0.805221,2023-03-24 14:12:09.306347,2023-03-24 14:13:03.545645,0 days 00:00:54.239298,ovo,1,linear,COMPLETE
7,7,0.805221,2023-03-24 14:12:09.307620,2023-03-24 14:13:03.484996,0 days 00:00:54.177376,ovo,3,linear,COMPLETE
8,8,0.805221,2023-03-24 14:13:03.441394,2023-03-24 14:14:02.071313,0 days 00:00:58.629919,ovo,2,linear,COMPLETE
9,9,0.800377,2023-03-24 14:13:03.494640,2023-03-24 14:14:00.159965,0 days 00:00:56.665325,ovo,1,sigmoid,COMPLETE


# Prediction analysis
This code creates a dataframe that includes the true labels, predicted labels, and predicted probabilities of the test set. It then adds the original text of each sentence in the test set to the dataframe. The code then selects the false positive predictions and ranks them by the probability of their positive prediction. The resulting dataframe is displayed, showing the top 10 false positives with the highest probability of being positive. Finally, the code selects the false negative predictions and ranks them by the probability of their negative prediction. The resulting dataframe is displayed, showing the top 10 false negatives with the lowest probability of being negative. This allows for an analysis of which types of sentences the model is misclassifying and may provide insights for improvement.


In [ ]:
# prediction analysis on test data
y_pred_proba = pipeline.predict_proba(x_test['text_pre'])
y_true = y_test['positive']
# create dataframe with true and predicted labels
df = pd.DataFrame({'y_true': y_true, 'y_pred': y_pred, 'y_pred_proba': y_pred_proba[:,1]})
# add text to dataframe
df['text'] = x_test['text']
#select only false positives and ranked by probability
false_positive_df = df[(df['y_true']==0) & (df['y_pred']==1)].sort_values('y_pred_proba', ascending=False)
display(false_positive_df.head(10))

#select only false negative and ranked by probability
false_negative_df = df[(df['y_true']==1) & (df['y_pred']==0)].sort_values('y_pred_proba', ascending=True)
display(false_negative_df.head(10))



,y_true,y_pred,y_pred_proba,text
286,0,1,0.958781,It's this pandering to the audience that sabot...
259,0,1,0.909585,This movie now joins Revenge of the Boogeyman ...
232,0,1,0.889589,The only consistent thread holding the series ...
58,0,1,0.875319,It defeats the purpose of a bluetooth headset.
244,0,1,0.865292,The directing and the cinematography aren't qu...
214,0,1,0.847603,The film is way too long.
27,0,1,0.847515,It's AGGRAVATING!
203,0,1,0.741901,"This movie suffered because of the writing, it..."
52,0,1,0.733195,"This item worked great, but it broke after 6 m..."
469,0,1,0.732823,"Maybe it's just their Vegetarian fare, but I'v..."


,y_true,y_pred,y_pred_proba,text
108,1,0,0.001259,"No shifting, no bubbling, no peeling, not even..."
596,1,0,0.008474,#NAME?
560,1,0,0.008474,#NAME?
551,1,0,0.010288,I went to Bachi Burger on a friend's recommend...
146,1,0,0.029905,"I've had no trouble accessing the Internet, do..."
136,1,0,0.059850,It plays louder than any other speaker of this...
181,1,0,0.070223,"They do not last forever, but is not overly ex..."
387,1,0,0.076660,A standout scene.
104,1,0,0.094006,Cheap but hey it works.. Was pleasantly supris...
573,1,0,0.099484,Seriously killer hot chai latte.


In [ ]:
_ = false_negative_df["text"].iloc[:10].apply(lambda x: print(x))

No shifting, no bubbling, no peeling, not even a scratch, NOTHING!I couldn't be more happier with my new one for the Droid.
#NAME?
#NAME?
I went to Bachi Burger on a friend's recommendation and was not disappointed.
I've had no trouble accessing the Internet, downloading ringtones or performing any of the functions.
It plays louder than any other speaker of this size; the price is so low that most would think the quality is lacking, however, it's not.
They do not last forever, but is not overly expensive to replace.Easy to operate and the sound is much better than others I have tried.
A standout scene.  
Cheap but hey it works.. Was pleasantly suprised given the low cost of this item.
Seriously killer hot chai latte.


In [ ]:
_ = false_positive_df["text"].iloc[:10].apply(lambda x: print(x))

It's this pandering to the audience that sabotages most of his films.  
This movie now joins Revenge of the Boogeyman and Zombiez as part of the hellish trinity of horror films.  
The only consistent thread holding the series together were the amazing performances of Leni Parker and Anita LaSelva as the two Taelons in quiet idealogical conflict.  
It defeats the purpose of a bluetooth headset.
The directing and the cinematography aren't quite as good.  
The film is way too long.  
It's AGGRAVATING!
This movie suffered because of the writing, it needed more suspense.  
This item worked great, but it broke after 6 months of use.
Maybe it's just their Vegetarian fare, but I've been twice and I thought it was average at best.
